In [ ]:
import pandas as pd
import ndjson
import ijson
import os
import praw
import multiprocessing

In [ ]:
dataset = pd.read_csv('dataset/dataset.csv', header=0)
#dataset = pd.read_csv('dataset/feb.csv', header=0)

In [ ]:
authors = dataset[['author_id','author']]
authors.drop_duplicates(inplace=True)
print(len(authors))
authors.head()

In [ ]:
def process_part(authors):
    
    reddit = praw.Reddit(client_id="2CkE8z4bPgIkhQ",
                     client_secret="fFIJX43pXxx8LvSQH4KAXMlZoXM",
                     user_agent="python:it.ppndrd.scraper:v1 (by u/ppndrd)")
    
    authors_dict = {}
    authors_dict["author_id"] = []
    authors_dict["author"] = []
    authors_dict["created_utc"] = []
    authors_dict["is_gold"] = []
    authors_dict["is_suspended"] = []
    authors_dict["karma"] = []

    for author in authors.itertuples():
        name = author[2]
        _id = author[1]
        redditor = reddit.redditor(name)

        try:
            redditor.id

            authors_dict["author_id"].append(_id)
            authors_dict["author"].append(name)

            try:
                authors_dict["is_suspended"].append(redditor.is_suspended)
                authors_dict["created_utc"].append(0)
                authors_dict["is_gold"].append(False)
                authors_dict["karma"].append(0)
            except:
                authors_dict["created_utc"].append(redditor.created_utc)
                authors_dict["is_gold"].append(redditor.is_gold)
                authors_dict["is_suspended"].append(False)
                authors_dict["karma"].append(redditor.link_karma)

        except Exception as e:
            pass
    
    return pd.DataFrame.from_dict(authors_dict)

In [ ]:
inputs = []
n = 915
inputs.append(authors[:10000])
for i in range(1,n-1):
    inputs.append(authors[i*10000:(i+1)*10000])
inputs.append(authors[(n-1)*10000:])

In [ ]:
outputs = []
p = multiprocessing.Pool(multiprocessing.cpu_count())
outputs = p.map(process_part, inputs)

In [ ]:
authors = pd.concat(outputs)
authors.to_csv('dataset/authors.csv', index=False)

In [ ]:
authors.head()

In [ ]:
dataset = pd.read_csv('dataset/dataset.csv', header=0)
authors = pd.read_csv('dataset/authors.csv', header=0)

In [ ]:
authors2 = authors.loc[authors['author_id'].isin(set(dataset['author_id']))]
print(len(authors2))

In [ ]:
authors2.to_csv('dataset/authors.csv')